In [ ]:
from model import *
from charset_normalizer import from_path
import torch
from model_copy import *
from torch.nn import functional
random.seed(0)
torch.manual_seed(0)

: 

In [14]:
in_channels = 3 
out_channels = 3
kernel_size = (2, 3)
x = torch.randn((1, in_channels , 32, 32))
target = torch.randn((1, 3 , 31, 30))

In [15]:
# With square kernels and equal stride
m = Conv2d(in_channels, out_channels, kernel_size)
ws,bs = m.param()
w = ws[0]
b = bs[0]
output_2 = m.forward(x)

# Output of PyTorch convolution
expected = functional.conv2d(x, w, b)
#expected = conv(x)

# Output of convolution as a matrix product
unfolded = torch.nn.functional.unfold(x, kernel_size=kernel_size)
wxb = w.view(out_channels, -1) @ unfolded + b.view(1, -1, 1)
actual = wxb.view(x.shape[0], out_channels, x.shape[2] - kernel_size[0] + 1, x.shape[3] - kernel_size[1]
+ 1)
torch.testing.assert_allclose(actual , expected)

torch.testing.assert_allclose(output_2 , expected)

In [20]:
target.shape

torch.Size([1, 3, 31, 30])

In [35]:
dl_grad = dMSE(output_2, target)
dl_grad.shape

torch.Size([1, 3, 31, 30])

In [33]:
w_grad, b_grad = m.backward(dl_grad)
print(w_grad, b_grad)

NotImplementedError: Input Error: Only unbatched (2D) or batched (3D) input Tensorsare supported (got 4D)

In [23]:
expected.weight.grad

AttributeError: 'Tensor' object has no attribute 'weight'